# Binding Prediction via API

http://tools.iedb.org/main/tools-api/

## MHC I

In [37]:
import requests
import csv

In [47]:
# HLA allele reference set (27 alleles long)
hla_ref_set = 'HLA-A*01:01,HLA-A*02:01,HLA-A*02:03,HLA-A*02:06,HLA-A*03:01,HLA-A*11:01,HLA-A*23:01,HLA-A*24:02,HLA-A*26:01,HLA-A*30:01,HLA-A*30:02,HLA-A*31:01,HLA-A*32:01,HLA-A*33:01,HLA-A*68:01,HLA-A*68:02,HLA-B*07:02,HLA-B*08:01,HLA-B*15:01,HLA-B*35:01,HLA-B*40:01,HLA-B*44:02,HLA-B*44:03,HLA-B*51:01,HLA-B*53:01,HLA-B*57:01,HLA-B*58:01'

# species must be set for each allele separately
species_set = 'human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human,human'

# length must be set for each allele separately
length_set = '9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9'




In [39]:
def read_data(filename):
    
    # read in single sequence
    with open(filename, mode='r') as read_file:
        return read_file.readline().strip('\n')
    
    
def predict_mhc1(a_set,s_set,l_set,sequence):
    
    # define post
    data = {'method':'recommended',
            'allele':a_set,
            'species':s_set,
            'length':l_set,
            'sequence_text':sequence}

    site = 'http://tools-cluster-interface.iedb.org/tools_api/mhci/'  

    # perform prediction
    return requests.post(site, data=data)

In [48]:
# perform prediction
sequence = read_data('data/Binding_Prediction/test_sequence.txt')
response = predict_mhc1(hla_ref_set,species_set,length_set,sequence)

In [49]:
# verify request was sucessful
response.status_code

200

In [50]:
# save into CSV format
write_filename = 'data/Binding_Prediction/test.csv'
with open(write_filename, mode='w') as write_file:
    file_writer = csv.writer(write_file, delimiter=',')
    
    for each_row in response.text.split('\n'):
        file_writer.writerow(each_row.split('\t'))